In [1]:
from pprint import pprint
from langchain_core.messages import AIMessage,HumanMessage

c:\Users\61068067\python_langgraph\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
messaages = [AIMessage(content=f"So you said you were seraching ocen mammals?",name="Model")]
messaages.append(HumanMessage(content=f"yes, that's right",name="Subhashini"))
messaages.append(AIMessage(content=f"Great, what would you like to learn about",name="Model"))
messaages.append(HumanMessage(content=f"I want to learn the best place to see Orcas in the US",name="Subhashini"))

for m in messaages:
    m.pretty_print()

================================== Ai Message ==================================
Name: Model

So you said you were seraching ocen mammals?
================================ Human Message =================================
Name: Subhashini

yes, that's right
================================== Ai Message ==================================
Name: Model

Great, what would you like to learn about
================================ Human Message =================================
Name: Subhashini

I want to learn the best place to see Orcas in the US


In [3]:
import os
import groq
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [4]:
llm=ChatGroq(model="qwen/qwen3-32b")
result=llm.invoke(messaages)
result

AIMessage(content="<think>\nOkay, the user wants to know the best places in the US to see orcas. Let me start by recalling where orcas are commonly found in the US. The most famous spot is the Pacific Northwest, especially around Washington State and British Columbia. That's where the Southern Resident Killer Whales are, right? They feed on salmon and are often seen near the San Juan Islands. I should mention that area first.\n\nBut wait, there are different types of orcas, like the Transient and Resident populations. The Southern Residents are the ones that are more social and easier to spot. Maybe the user isn't aware of the different types, so explaining that briefly could add context.\n\nThen there's Northern Washington and Alaska. Alaska has a lot of orca activity too, especially in places like Prince William Sound and the Inside Passage. The Northern Residents there are similar to the Southern ones, but maybe they have different behaviors. Also, the Transients in Alaska might be 

In [6]:
def multiply(a:int,b:int)->int:
    """Multiply a and b.
    Agrs:
    a:first int
    b:sercond int
    """
    return a*b

In [7]:
llm_with_tools = llm.bind_tools([multiply])

In [8]:
tool_call=llm_with_tools.invoke("What is 2 multiplied by 3")
tool_call.tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': '7rkaq55rw',
  'type': 'tool_call'}]

In [14]:
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.graph.message import AnyMessage,add_messages

In [ ]:
class MessageState(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]

In [10]:
from langgraph.graph import MessagesState

class MessageState(MessagesState):
    pass

In [11]:
initial_message=[AIMessage(content=f"Hello, How can I assit you?",name="Model"),
                 HumanMessage(content=f"I am looking for information on marine biology",name="Subhashini")]

In [12]:
new_message=AIMessage(content=f"Sure, I can help with that.What specifically are you interested in?",name="Model")

In [15]:
add_messages(initial_message,new_message)

[AIMessage(content='Hello, How can I assit you?', additional_kwargs={}, response_metadata={}, name='Model', id='a51e3f03-e9b6-4238-8e25-429f7f04f86c', tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='I am looking for information on marine biology', additional_kwargs={}, response_metadata={}, name='Subhashini', id='8fe31801-f40a-4295-b429-9593392a0bde'),
 AIMessage(content='Sure, I can help with that.What specifically are you interested in?', additional_kwargs={}, response_metadata={}, name='Model', id='12f4855d-9b1a-4e0b-8e1a-a3b86416da8c', tool_calls=[], invalid_tool_calls=[])]

In [17]:
from langgraph.graph import StateGraph,START,END

def tool_calling_llm(state:MessagesState):
    return {"messages":[llm_with_tools.invoke(state["messages"])]}

In [18]:
builder = StateGraph(MessagesState)

builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_edge(START,"tool_calling_llm")
builder.add_edge("tool_calling_llm",END)

graph=builder.compile()


In [19]:
messages = graph.invoke({"messages":HumanMessage(content=f"Multiply 2 and 3")})

for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Multiply 2 and 3
================================== Ai Message ==================================
Tool Calls:
  multiply (nr70je25v)
 Call ID: nr70je25v
  Args:
    a: 2
    b: 3
